# Industrial Safety Equipment Detection - Inference Notebook

This notebook demonstrates how to use the trained YOLOv8 model for detecting industrial safety equipment (FireExtinguisher, ToolBox, OxygenTank) in images.

## Features:
- Load trained YOLOv8 model
- Run inference on single images or batch processing
- Visualize detection results
- Export results in various formats
- Performance benchmarking

## 1. Setup and Imports

In [ ]:
# Install required packages in Colab
import os
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install ultralytics
    !pip install opencv-python
    !pip install pillow
    print("Packages installed successfully!")
else:
    print("Not running in Colab - assuming packages are already installed")

In [ ]:
import os
import logging
from pathlib import Path
import time
from typing import List, Tuple, Optional

import cv2
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ultralytics import YOLO
from IPython.display import display, Image as IPImage
import pandas as pd

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Configuration

In [ ]:
# Configuration
CONFIG = {
    'model_path': 'runs/train/industrial_safety_detection/weights/best.pt',  # Path to trained model
    'conf_threshold': 0.25,
    'iou_threshold': 0.45,
    'device': 'auto',  # 'auto', 'cpu', 'cuda', or specific GPU ID
    'class_names': ['FireExtinguisher', 'ToolBox', 'OxygenTank'],
    'colors': [(255, 0, 0), (0, 255, 0), (0, 0, 255)],  # BGR colors for each class
    'output_dir': 'inference_results'
}

# Create output directory
os.makedirs(CONFIG['output_dir'], exist_ok=True)

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 3. Load Trained Model

In [ ]:
# Check if model exists
model_path = CONFIG['model_path']
if not os.path.exists(model_path):
    print(f"⚠ Model not found at: {model_path}")
    print("Please make sure you have trained the model first or provide the correct path.")
    print("\nAlternatively, you can use a pretrained YOLO model:")
    model_path = 'yolov8n.pt'  # Use pretrained model as fallback
    print(f"Using pretrained model: {model_path}")
else:
    print(f"✓ Model found: {model_path}")

# Load model
try:
    model = YOLO(model_path)
    print(f"✓ Model loaded successfully")
    
    # Model info
    print(f"\nModel Information:")
    model.info()
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

## 4. Utility Functions

In [ ]:
def visualize_predictions(image_path, results, class_names, colors, save_path=None):
    """
    Visualize detection results on an image.
    
    Args:
        image_path: Path to the input image
        results: YOLO results object
        class_names: List of class names
        colors: List of colors for each class
        save_path: Optional path to save the visualization
    """
    # Load image
    image = cv2.imread(str(image_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Create figure
    plt.figure(figsize=(12, 8))
    plt.imshow(image_rgb)
    
    # Get current axes
    ax = plt.gca()
    
    # Process detections
    if len(results) > 0 and len(results[0].boxes) > 0:
        boxes = results[0].boxes
        
        for box in boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0])
            class_id = int(box.cls[0])
            
            # Get class name and color
            if class_id < len(class_names):
                class_name = class_names[class_id]
                color = np.array(colors[class_id]) / 255.0  # Normalize for matplotlib
            else:
                class_name = f"Class_{class_id}"
                color = (1.0, 1.0, 1.0)  # White for unknown classes
            
            # Draw bounding box
            rect = patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=2, edgecolor=color, facecolor='none'
            )
            ax.add_patch(rect)
            
            # Add label
            label = f"{class_name}: {conf:.2f}"
            ax.text(
                x1, y1 - 10, label,
                bbox=dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.7),
                fontsize=10, color='white', weight='bold'
            )
    
    plt.axis('off')
    plt.title(f'Industrial Safety Equipment Detection\nImage: {Path(image_path).name}', 
              fontsize=14, fontweight='bold')
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        print(f"Visualization saved to: {save_path}")
    
    plt.show()

def get_detection_summary(results, class_names):
    """
    Get summary of detections.
    
    Args:
        results: YOLO results object
        class_names: List of class names
        
    Returns:
        Dictionary with detection summary
    """
    summary = {
        'total_detections': 0,
        'class_counts': {name: 0 for name in class_names},
        'confidences': [],
        'detections': []
    }
    
    if len(results) > 0 and len(results[0].boxes) > 0:
        boxes = results[0].boxes
        summary['total_detections'] = len(boxes)
        
        for box in boxes:
            conf = float(box.conf[0])
            class_id = int(box.cls[0])
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            
            if class_id < len(class_names):
                class_name = class_names[class_id]
                summary['class_counts'][class_name] += 1
            else:
                class_name = f"Unknown_{class_id}"
                if class_name not in summary['class_counts']:
                    summary['class_counts'][class_name] = 0
                summary['class_counts'][class_name] += 1
            
            summary['confidences'].append(conf)
            summary['detections'].append({
                'class': class_name,
                'confidence': conf,
                'bbox': [x1, y1, x2, y2]
            })
    
    return summary

print("✓ Utility functions defined")

## 5. Single Image Inference

In [ ]:
# Single image inference example
def run_single_inference(image_path, model, config):
    """
    Run inference on a single image.
    
    Args:
        image_path: Path to the input image
        model: Loaded YOLO model
        config: Configuration dictionary
    """
    if not os.path.exists(image_path):
        print(f"❌ Image not found: {image_path}")
        return
    
    print(f"Running inference on: {image_path}")
    
    # Record inference time
    start_time = time.time()
    
    # Run inference
    results = model(
        image_path,
        conf=config['conf_threshold'],
        iou=config['iou_threshold'],
        device=config['device']
    )
    
    inference_time = time.time() - start_time
    
    # Get detection summary
    summary = get_detection_summary(results, config['class_names'])
    
    # Print results
    print(f"\n📊 Inference Results:")
    print(f"⏱ Inference time: {inference_time:.3f} seconds")
    print(f"🎯 Total detections: {summary['total_detections']}")
    
    if summary['total_detections'] > 0:
        print(f"\n📈 Class Distribution:")
        for class_name, count in summary['class_counts'].items():
            if count > 0:
                print(f"  {class_name}: {count}")
        
        avg_conf = np.mean(summary['confidences'])
        print(f"\n🎯 Average confidence: {avg_conf:.3f}")
        
        print(f"\n📋 Detailed detections:")
        for i, det in enumerate(summary['detections']):
            print(f"  {i+1}. {det['class']}: {det['confidence']:.3f}")
    
    # Visualize results
    output_path = os.path.join(config['output_dir'], f"result_{Path(image_path).stem}.jpg")
    visualize_predictions(image_path, results, config['class_names'], config['colors'], output_path)
    
    return results, summary

# Example usage - replace with your image path
sample_image_path = "data/test/images/000000000.png"  # Update this path

if os.path.exists(sample_image_path):
    results, summary = run_single_inference(sample_image_path, model, CONFIG)
else:
    print(f"Sample image not found: {sample_image_path}")
    print("Please provide a valid image path in the next cell.")

In [ ]:
# Upload and test your own image
# In Colab, you can use this code to upload an image:

if IN_COLAB:
    from google.colab import files
    print("Upload an image file:")
    uploaded = files.upload()
    
    if uploaded:
        # Get the first uploaded file
        filename = list(uploaded.keys())[0]
        print(f"\nUploaded: {filename}")
        
        # Run inference on uploaded image
        results, summary = run_single_inference(filename, model, CONFIG)
else:
    # For local execution, specify your image path
    custom_image_path = input("Enter path to your image (or press Enter to skip): ")
    
    if custom_image_path and os.path.exists(custom_image_path):
        results, summary = run_single_inference(custom_image_path, model, CONFIG)
    else:
        print("No valid image path provided or file not found.")

## 6. Batch Processing

In [ ]:
def run_batch_inference(images_dir, model, config, max_images=10):
    """
    Run inference on multiple images in a directory.
    
    Args:
        images_dir: Directory containing images
        model: Loaded YOLO model
        config: Configuration dictionary
        max_images: Maximum number of images to process
    """
    images_dir = Path(images_dir)
    if not images_dir.exists():
        print(f"❌ Directory not found: {images_dir}")
        return
    
    # Find all image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    image_files = []
    for ext in image_extensions:
        image_files.extend(list(images_dir.glob(f'*{ext}')))
        image_files.extend(list(images_dir.glob(f'*{ext.upper()}')))
    
    if not image_files:
        print(f"❌ No image files found in: {images_dir}")
        return
    
    # Limit number of images
    image_files = image_files[:max_images]
    print(f"Processing {len(image_files)} images...")
    
    # Batch processing results
    batch_results = []
    total_detections = 0
    total_time = 0
    class_totals = {name: 0 for name in config['class_names']}
    
    for i, image_path in enumerate(image_files):
        print(f"\n[{i+1}/{len(image_files)}] Processing: {image_path.name}")
        
        try:
            # Run inference
            start_time = time.time()
            results = model(
                str(image_path),
                conf=config['conf_threshold'],
                iou=config['iou_threshold'],
                device=config['device'],
                verbose=False
            )
            inference_time = time.time() - start_time
            total_time += inference_time
            
            # Get summary
            summary = get_detection_summary(results, config['class_names'])
            total_detections += summary['total_detections']
            
            # Update class totals
            for class_name, count in summary['class_counts'].items():
                if class_name in class_totals:
                    class_totals[class_name] += count
            
            # Store results
            batch_results.append({
                'image': image_path.name,
                'detections': summary['total_detections'],
                'time': inference_time,
                'summary': summary
            })
            
            print(f"  ✓ {summary['total_detections']} detections in {inference_time:.3f}s")
            
        except Exception as e:
            print(f"  ❌ Error processing {image_path.name}: {e}")
            continue
    
    # Print batch summary
    print(f"\n{'='*60}")
    print(f"📊 BATCH PROCESSING SUMMARY")
    print(f"{'='*60}")
    print(f"Images processed: {len(batch_results)}")
    print(f"Total detections: {total_detections}")
    print(f"Total time: {total_time:.2f}s")
    print(f"Average time per image: {total_time/len(batch_results):.3f}s")
    print(f"Average detections per image: {total_detections/len(batch_results):.2f}")
    
    print(f"\n📈 Class Distribution:")
    for class_name, count in class_totals.items():
        if count > 0:
            print(f"  {class_name}: {count}")
    
    return batch_results

# Run batch inference on test images
test_images_dir = "data/test/images"
if os.path.exists(test_images_dir):
    batch_results = run_batch_inference(test_images_dir, model, CONFIG, max_images=5)
else:
    print(f"Test images directory not found: {test_images_dir}")
    print("Please provide a directory containing images for batch processing.")

## 7. Performance Benchmarking

In [ ]:
def benchmark_model(model, config, num_runs=10, image_size=(640, 640)):
    """
    Benchmark model performance.
    
    Args:
        model: Loaded YOLO model
        config: Configuration dictionary
        num_runs: Number of benchmark runs
        image_size: Input image size (width, height)
    """
    print(f"🚀 Benchmarking model performance...")
    print(f"Runs: {num_runs}, Image size: {image_size}")
    
    # Create dummy image
    dummy_image = np.random.randint(0, 255, (image_size[1], image_size[0], 3), dtype=np.uint8)
    
    # Warm up
    print("Warming up...")
    for _ in range(3):
        _ = model(dummy_image, verbose=False)
    
    # Benchmark
    times = []
    for i in range(num_runs):
        start_time = time.time()
        _ = model(
            dummy_image,
            conf=config['conf_threshold'],
            iou=config['iou_threshold'],
            device=config['device'],
            verbose=False
        )
        end_time = time.time()
        times.append(end_time - start_time)
        
        if (i + 1) % 5 == 0:
            print(f"  Completed {i + 1}/{num_runs} runs")
    
    # Calculate statistics
    avg_time = np.mean(times)
    std_time = np.std(times)
    min_time = np.min(times)
    max_time = np.max(times)
    fps = 1.0 / avg_time
    
    print(f"\n📊 BENCHMARK RESULTS")
    print(f"{'='*40}")
    print(f"Average inference time: {avg_time:.4f}s ± {std_time:.4f}s")
    print(f"Min inference time: {min_time:.4f}s")
    print(f"Max inference time: {max_time:.4f}s")
    print(f"Average FPS: {fps:.2f}")
    print(f"Device: {config['device']}")
    print(f"Image size: {image_size}")
    
    # Plot timing distribution
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.hist(times, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    plt.xlabel('Inference Time (seconds)')
    plt.ylabel('Frequency')
    plt.title('Inference Time Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(times) + 1), times, marker='o', linewidth=2, markersize=4)
    plt.axhline(y=avg_time, color='red', linestyle='--', label=f'Average: {avg_time:.4f}s')
    plt.xlabel('Run Number')
    plt.ylabel('Inference Time (seconds)')
    plt.title('Inference Time per Run')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'avg_time': avg_time,
        'std_time': std_time,
        'min_time': min_time,
        'max_time': max_time,
        'fps': fps,
        'times': times
    }

# Run benchmark
benchmark_results = benchmark_model(model, CONFIG, num_runs=10)

## 8. Export Results

In [ ]:
def export_results_to_csv(batch_results, output_path="inference_results.csv"):
    """
    Export batch inference results to CSV.
    
    Args:
        batch_results: Results from batch inference
        output_path: Output CSV file path
    """
    if not batch_results:
        print("No batch results to export.")
        return
    
    # Prepare data for CSV
    csv_data = []
    
    for result in batch_results:
        base_row = {
            'image': result['image'],
            'total_detections': result['detections'],
            'inference_time': result['time']
        }
        
        # Add class counts
        for class_name in CONFIG['class_names']:
            base_row[f'{class_name}_count'] = result['summary']['class_counts'].get(class_name, 0)
        
        # If there are detections, add detailed info
        if result['summary']['detections']:
            for i, detection in enumerate(result['summary']['detections']):
                row = base_row.copy()
                row.update({
                    'detection_id': i + 1,
                    'class': detection['class'],
                    'confidence': detection['confidence'],
                    'bbox_x1': detection['bbox'][0],
                    'bbox_y1': detection['bbox'][1],
                    'bbox_x2': detection['bbox'][2],
                    'bbox_y2': detection['bbox'][3]
                })
                csv_data.append(row)
        else:
            # No detections
            base_row.update({
                'detection_id': 0,
                'class': 'None',
                'confidence': 0.0,
                'bbox_x1': 0,
                'bbox_y1': 0,
                'bbox_x2': 0,
                'bbox_y2': 0
            })
            csv_data.append(base_row)
    
    # Create DataFrame and save
    df = pd.DataFrame(csv_data)
    df.to_csv(output_path, index=False)
    
    print(f"✓ Results exported to: {output_path}")
    print(f"Total rows: {len(df)}")
    
    # Display first few rows
    print("\nFirst 5 rows:")
    display(df.head())
    
    return df

# Export results if batch processing was done
if 'batch_results' in locals() and batch_results:
    results_df = export_results_to_csv(batch_results, 
                                      os.path.join(CONFIG['output_dir'], "inference_results.csv"))
else:
    print("No batch results available for export.")

## 9. Model Analysis and Statistics

In [ ]:
# Model analysis
def analyze_model_performance():
    """
    Analyze model performance and provide insights.
    """
    print("🔍 MODEL ANALYSIS")
    print("="*50)
    
    # Basic model info
    print(f"Model architecture: {CONFIG['model_path']}")
    print(f"Classes: {', '.join(CONFIG['class_names'])}")
    print(f"Confidence threshold: {CONFIG['conf_threshold']}")
    print(f"IoU threshold: {CONFIG['iou_threshold']}")
    
    # Performance metrics
    if 'benchmark_results' in locals():
        print(f"\n⚡ PERFORMANCE:")
        print(f"Average inference time: {benchmark_results['avg_time']:.4f}s")
        print(f"Average FPS: {benchmark_results['fps']:.2f}")
        
        # Performance rating
        if benchmark_results['fps'] > 30:
            perf_rating = "🚀 Excellent (Real-time capable)"
        elif benchmark_results['fps'] > 15:
            perf_rating = "⚡ Good (Near real-time)"
        elif benchmark_results['fps'] > 5:
            perf_rating = "⚠ Moderate (Batch processing suitable)"
        else:
            perf_rating = "🐌 Slow (Consider optimization)"
        
        print(f"Performance rating: {perf_rating}")
    
    # Detection statistics
    if 'batch_results' in locals() and batch_results:
        print(f"\n📊 DETECTION STATISTICS:")
        total_images = len(batch_results)
        total_detections = sum(r['detections'] for r in batch_results)
        images_with_detections = sum(1 for r in batch_results if r['detections'] > 0)
        
        print(f"Images processed: {total_images}")
        print(f"Total detections: {total_detections}")
        print(f"Images with detections: {images_with_detections} ({images_with_detections/total_images*100:.1f}%)")
        print(f"Average detections per image: {total_detections/total_images:.2f}")
        
        if images_with_detections > 0:
            avg_det_per_positive = total_detections / images_with_detections
            print(f"Average detections per positive image: {avg_det_per_positive:.2f}")
    
    print(f"\n💡 RECOMMENDATIONS:")
    if 'benchmark_results' in locals():
        if benchmark_results['fps'] < 10:
            print("  • Consider using a smaller model (YOLOv8n → YOLOv8s) for better speed")
            print("  • Try reducing input image size")
            print("  • Enable half precision (FP16) if using GPU")
        
        if benchmark_results['std_time'] > benchmark_results['avg_time'] * 0.1:
            print("  • High variance in inference time - consider model optimization")
    
    print("  • Monitor confidence scores - low scores may indicate need for more training data")
    print("  • Consider ensemble methods for improved accuracy")
    print("  • Implement confidence thresholding based on your use case requirements")

# Run analysis
analyze_model_performance()

## 10. Summary and Conclusion

In [ ]:
print("\n" + "="*60)
print("🎯 INFERENCE SESSION SUMMARY")
print("="*60)

print(f"Model: {CONFIG['model_path']}")
print(f"Classes: {', '.join(CONFIG['class_names'])}")
print(f"Device: {CONFIG['device']}")

if 'benchmark_results' in locals():
    print(f"\n⚡ Performance:")
    print(f"  Average inference time: {benchmark_results['avg_time']:.4f}s")
    print(f"  Average FPS: {benchmark_results['fps']:.2f}")

if 'batch_results' in locals() and batch_results:
    total_processed = len(batch_results)
    total_detections = sum(r['detections'] for r in batch_results)
    print(f"\n📊 Batch Processing:")
    print(f"  Images processed: {total_processed}")
    print(f"  Total detections: {total_detections}")

print(f"\n📁 Output Directory: {CONFIG['output_dir']}")
print(f"  - Visualization images")
if 'results_df' in locals():
    print(f"  - Results CSV file")

print(f"\n🚀 Next Steps:")
print(f"  1. Review detection results and adjust confidence thresholds if needed")
print(f"  2. Test on more diverse images to evaluate robustness")
print(f"  3. Consider model optimization for deployment")
print(f"  4. Implement the model in your production pipeline")

print("\n✅ Inference session completed successfully!")
print("="*60)